In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp data

# Data
> Utilies for generating time series datasets

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
import numpy as np
import pandas as pd

from utilsforecast.compat import DataFrame

In [ ]:
#| export
def generate_series(
    n_series: int,
    freq: str = 'D',
    min_length: int = 50,
    max_length: int = 500,
    n_static_features: int = 0,
    equal_ends: bool = False,
    with_trend: bool = False,
    engine: str = 'pandas',
    seed: int = 0,
) -> DataFrame:
    """Generate Synthetic Panel Series.

    Parameters
    ----------
    n_series : int
        Number of series for synthetic panel.
    freq : str (default='D')
        Frequency of the data, 'D' or 'M'.
    min_length : int (default=50)
        Minimum length of synthetic panel's series.
    max_length : int (default=500)
        Maximum length of synthetic panel's series.
    n_static_features : int (default=0)
        Number of static exogenous variables for synthetic panel's series.
    equal_ends : bool (default=False)
        Series should end in the same date stamp `ds`.
    with_trend : bool (default=False)
        Series should have a (positive) trend.
    engine : str (default='pandas')
        Output Dataframe type.        
    seed : int (default=0)
        Random seed used for generating the data.

    Returns
    -------
    series : pandas or polars DataFrame
        Synthetic panel with columns [`unique_id`, `ds`, `y`] and exogenous features.
    """
    available_engines = ['pandas', 'polars']
    engine = engine.lower()
    if engine not in available_engines:
        raise ValueError(
            f"{engine} is not a correct engine; available options: {available_engines}"
        )
    seasonalities = {'D': 7, 'M': 12}        
    available_frequencies = seasonalities.keys()
    if freq not in available_frequencies:
        raise ValueError(f"Currently soported frequencies are: {available_frequencies}, got {freq}")
        
    rng = np.random.RandomState(seed)
    series_lengths = rng.randint(min_length, max_length + 1, n_series)
    total_length = series_lengths.sum()

    season = seasonalities[freq]    
    vals_dict = {
        'unique_id': np.repeat(np.arange(n_series), series_lengths),
        'y': np.arange(total_length) % season + rng.rand(total_length) * 0.5
    }

    dates = pd.date_range('2000-01-01', periods=max_length, freq=freq).values
    if equal_ends:
        series_dates = [dates[-length:] for length in series_lengths]
    else:
        series_dates = [dates[:length] for length in series_lengths]
    vals_dict['ds'] = np.concatenate(series_dates)

    for i in range(n_static_features):
        static_values = np.repeat(rng.randint(0, 100, n_series), series_lengths)
        vals_dict[f'static_{i}'] = static_values
        if i == 0:
            vals_dict['y'] = vals_dict['y'] * (1 + vals_dict[f'static_{i}'])

    if with_trend:
        coefs = np.repeat(rng.rand(n_series), series_lengths)
        trends = np.concatenate([np.arange(length) for length in series_lengths])
        vals_dict['y'] += coefs * trends

    cat_cols = [col for col in vals_dict.keys() if 'static' in col]
    cat_cols.append('unique_id')
    if engine == 'pandas':
        df = pd.DataFrame(vals_dict)
        df[cat_cols] = df[cat_cols].astype('category')
        df['unique_id'] = df['unique_id'].cat.as_ordered()
    else:
        import polars as pl

        df = pl.DataFrame(vals_dict)
        df = df.with_columns(pl.col('unique_id').sort())
        for col in cat_cols:
            df = df.with_columns(pl.col(col).cast(str).cast(pl.Categorical))
    return df

In [ ]:
show_doc(generate_series)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/data.py#L13){target="_blank" style="float:right; font-size:smaller"}

### generate_series

>      generate_series (n_series:int, freq:str='D', min_length:int=50,
>                       max_length:int=500, n_static_features:int=0,
>                       equal_ends:bool=False, with_trend:bool=False,
>                       engine:str='pandas', seed:int=0)

Generate Synthetic Panel Series.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| n_series | int |  | Number of series for synthetic panel. |
| freq | str | D | Frequency of the data, 'D' or 'M'. |
| min_length | int | 50 | Minimum length of synthetic panel's series. |
| max_length | int | 500 | Maximum length of synthetic panel's series. |
| n_static_features | int | 0 | Number of static exogenous variables for synthetic panel's series. |
| equal_ends | bool | False | Series should end in the same date stamp `ds`. |
| with_trend | bool | False | Series should have a (positive) trend. |
| engine | str | pandas | Output Dataframe type.         |
| seed | int | 0 | Random seed used for generating the data. |
| **Returns** | **Union** |  | **Synthetic panel with columns [`unique_id`, `ds`, `y`] and exogenous features.** |

In [ ]:
synthetic_panel = generate_series(n_series=2)
synthetic_panel.groupby('unique_id').head(4)

,unique_id,y,ds
0,0,0.357595,2000-01-01
1,0,1.301382,2000-01-02
2,0,2.272442,2000-01-03
3,0,3.211827,2000-01-04
222,1,5.399023,2000-01-01
223,1,6.092818,2000-01-02
224,1,0.476396,2000-01-03
225,1,1.343744,2000-01-04
